# Alternating Human/AI messages for CHAT

In this strategy, few-shot prompting utilizes alternating human and AI messages. This technique can be especially beneficial for chat-oriented applications since the language model must comprehend the conversational context and provide appropriate responses.

While this approach effectively handles conversation context and is easy to implement for chat-based applications, it lacks flexibility for other application types and is limited to chat-based models. However, we can use alternating human/AI messages to create a chat prompt that translates English into pirate language. 

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv, find_dotenv
import os

In [2]:
load_dotenv(find_dotenv('newenv.env'))

True

In [3]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

C:\Users\Vaibhav\AppData\Local\Temp\ipykernel_22680\895470827.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [4]:
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")

system_template = "You are a helpful assisant for converting english queries into pirate."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "{text}"
human_message_prompt  = HumanMessagePromptTemplate.from_template(human_template)

In [6]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=llm, prompt = chat_prompt)
chain.run("I love programming")

"I be lovin' the art o' codin', aye!"

In [7]:
example_human

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hi'), additional_kwargs={})

In [8]:
example_ai

AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Argh me mateys'), additional_kwargs={})

In [9]:
system_message_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assisant for converting english queries into pirate.'), additional_kwargs={})

In [10]:
human_message_prompt

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})

In [11]:
chat_prompt

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assisant for converting english queries into pirate.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hi'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Argh me mateys'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

# Few-shot prompting

Few-shot prompting can lead to improved output quality because the model can learn the task better by observing the examples. However, the increased token usage may worsen the results if the examples are not well chosen or are misleading.

In [12]:
from langchain import FewShotPromptTemplate, PromptTemplate

examples = [
    {
        "query":"",
        "answer":""
    },
    {
        "query":"",
        "answer":""
    },
    {
        "query":"",
        "answer":""
    },
    {
        "query":"",
        "answer":""
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables = ["query","answer"],
    template = example_template
    )

prefix = """
The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: 
"""

few_shot_prompt_template = FewShotPromptTemplate(
examples = examples,
example_prompt = example_prompt,
prefix = prefix,
suffix= suffix,
input_variables = ["query"],
example_separator = "\n\n"    
)

In [13]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)
chain.run("What's the secret to happiness?")

'The secret to happiness? Well, it’s a mix of chocolate, a good Wi-Fi connection, and the occasional dance party in your living room. Just remember, if you can’t find happiness, try looking under the couch cushions—sometimes it’s hiding with the remote!'

# Example Selectors

Example selectors can be used to provide a few-shot learning experience. The primary goal of few-shot learning is to learn a similarity function that maps the similarities between classes in the support and query sets. In this context, an example selector can be designed to choose a set of relevant examples that are representative of the desired output.

The **LengthBasedExampleSelector** is useful when you're concerned about the length of the context window. It selects fewer examples for longer queries and more examples for shorter queries.

In [14]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [15]:
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

example_template = """
Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template # or template = """Word: {word}\nAntonym: {antonym}"""
)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,
)

In [16]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {word}\nAntonym:",
    input_variables=["word"],
    example_separator="\n\n",
)

**We can generate a prompt using the format method for understanding intermediate functionality**

In [17]:
print(dynamic_prompt.format(word="big"))

Give the antonym of every input


Word: happy
Antonym: sad



Word: tall
Antonym: short



Word: energetic
Antonym: lethargic



Word: sunny
Antonym: gloomy


Word: big
Antonym:


This method is effective for managing a large number of examples. It offers customization through various selectors, but it involves manual creation and selection of examples, which might not be ideal for every application type.

Example of employing LangChain's **SemanticSimilarityExampleSelector** for selecting examples based on their semantic resemblance to the input. This illustration showcases the process of creating an ExampleSelector, generating a prompt using a few-shot approach:

In [18]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings import OpenAIEmbeddings

In [19]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Define some examples
examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

In [20]:
# create Deep Lake dataset locally
db = DeepLake(dataset_path='./mydb/')

In [22]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings, OpenAIEmbeddings

# hf = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en", model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True})
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [23]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, embeddings, db, k=1
)

# Create a FewShotPromptTemplate using the example_selector
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Convert the temperature from Celsius to Fahrenheit",
    suffix="Input: {temperature}\nOutput:", 
    input_variables=["temperature"],
)

Creating 5 embeddings in 1 batches of size 5:: 100%|██████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.38s/it]

Dataset(path='./deeplake/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (5, 1)      str     None   
 metadata     json      (5, 1)      str     None   
 embedding  embedding  (5, 1536)  float32   None   
    id        text      (5, 1)      str     None   


In [24]:
# Test the similar_prompt with different inputs
print(similar_prompt.format(temperature="10°C"))   # Test with an input
print(similar_prompt.format(temperature="30°C"))  # Test with another input

Convert the temperature from Celsius to Fahrenheit

Input: 10°C
Output: 50°F

Input: 10°C
Output:
Convert the temperature from Celsius to Fahrenheit

Input: 30°C
Output: 86°F

Input: 30°C
Output:


In [25]:
# Add a new example to the SemanticSimilarityExampleSelector
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(similar_prompt.format(temperature="40°C")) # Test with a new input after adding the example

Creating 1 embeddings in 1 batches of size 1:: 100%|██████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.61it/s]


Dataset(path='./deeplake/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (6, 1)      str     None   
 metadata     json      (6, 1)      str     None   
 embedding  embedding  (6, 1536)  float32   None   
    id        text      (6, 1)      str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 40°C
Output: 104°F

Input: 40°C
Output:
